In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold, learning_curve
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import os # For saving deployment files

warnings.filterwarnings('ignore')

## 1. Data Loading and Initial Preparation

csv_path = '/content/local_authority_traffic (1).csv'

try:
    df = pd.read_csv(csv_path)
    print(f"Traffic data loaded: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: File not found at: {csv_path}")
    raise

print("\nInitial Data Overview")
# ... (removed print statements for brevity)

# 2. Train/Test Split First (Prevent Leakage)
print("\n## 2. Train/Test Split (Preventing Leakage)")

# Drop columns we won't use
df_clean = df.drop(columns=['link_length_miles', 'cars_and_taxis', 'local_authority_name'])

# Separate target
y = df_clean['all_motor_vehicles']
X = df_clean.drop(columns=['all_motor_vehicles'])

# SPLIT FIRST
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"Train set: {X_train.shape[0]} samples, Test set: {X_test.shape[0]} samples")

## 3. Feature Engineering (Training Data Only)
print("\n## 3. Feature Engineering and Encoding")

X_train_eng = X_train.copy()
X_test_eng = X_test.copy()

# Temporal Features
print("1. Creating temporal features...")
min_year_train = X_train_eng['year'].min()
max_year_train = X_train_eng['year'].max()

for df_split in [X_train_eng, X_test_eng]:
    df_split['year_squared'] = df_split['year'] ** 2
    df_split['years_since_start'] = df_split['year'] - min_year_train
    df_split['year_sin'] = np.sin(2 * np.pi * df_split['year'] / max_year_train)
    df_split['year_cos'] = np.cos(2 * np.pi * df_split['year'] / max_year_train)

# Road Length Features
print("2. Creating road length features and bins...")
for df_split in [X_train_eng, X_test_eng]:
    df_split['link_length_km_squared'] = df_split['link_length_km'] ** 2
    df_split['link_length_log'] = np.log1p(df_split['link_length_km'])
    df_split['link_length_inv'] = 1 / (df_split['link_length_km'] + 0.1)

# Road length bins (based on training data quantiles)
length_bins = [0] + X_train_eng['link_length_km'].quantile([0.33, 0.67]).tolist() + [float('inf')]
X_train_eng['road_length_category'] = pd.cut(X_train_eng['link_length_km'], bins=length_bins, labels=['short', 'medium', 'long'])
X_test_eng['road_length_category'] = pd.cut(X_test_eng['link_length_km'], bins=length_bins, labels=['short', 'medium', 'long'])

# Interaction Features
print("3. Creating interaction features...")
for df_split in [X_train_eng, X_test_eng]:
    df_split['year_x_length'] = df_split['year'] * df_split['link_length_km']
    df_split['year_x_length_sq'] = df_split['year'] * df_split['link_length_km_squared']

# Authority Frequency Features
print("4. Creating authority frequency features...")
authority_counts = X_train_eng['local_authority_id'].value_counts().to_dict()
median_authority_frequency = X_train_eng['local_authority_id'].map(authority_counts).median()

X_train_eng['authority_frequency'] = X_train_eng['local_authority_id'].map(authority_counts)
X_test_eng['authority_frequency'] = X_test_eng['local_authority_id'].map(authority_counts)
X_test_eng['authority_frequency'].fillna(median_authority_frequency, inplace=True)

X_train_eng['authority_log_freq'] = np.log1p(X_train_eng['authority_frequency'])
X_test_eng['authority_log_freq'] = np.log1p(X_test_eng['authority_frequency'])

# Target Encoding (with CV and Strong Regularization)
print("5. Applying target encoding (CV + Smoothing)...")
smoothing_factor = 50
global_mean = y_train.mean()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
X_train_eng['authority_target_enc'] = 0.0

for train_idx, val_idx in kf.split(X_train_eng):
    fold_data = X_train_eng.iloc[train_idx].copy()
    fold_data['target'] = y_train.iloc[train_idx].values
    fold_encoding = fold_data.groupby('local_authority_id')['target'].agg(['mean', 'count'])
    fold_encoding['smoothed_mean'] = ((fold_encoding['mean'] * fold_encoding['count'] + global_mean * smoothing_factor) /
                                      (fold_encoding['count'] + smoothing_factor))
    X_train_eng.loc[X_train_eng.index[val_idx], 'authority_target_enc'] = X_train_eng.iloc[val_idx]['local_authority_id'].map(fold_encoding['smoothed_mean'])
X_train_eng['authority_target_enc'].fillna(global_mean, inplace=True)

test_data = X_train_eng.copy()
test_data['target'] = y_train.values
test_encoding = test_data.groupby('local_authority_id')['target'].agg(['mean', 'count'])
test_encoding['smoothed_mean'] = ((test_encoding['mean'] * test_encoding['count'] + global_mean * smoothing_factor) /
                                  (test_encoding['count'] + smoothing_factor))
X_test_eng['authority_target_enc'] = X_test_eng['local_authority_id'].map(test_encoding['smoothed_mean'])
X_test_eng['authority_target_enc'].fillna(global_mean, inplace=True)

# Encode Categoricals and Final Cleanup
print("6. Final encoding and cleanup...")
X_train_eng = pd.get_dummies(X_train_eng, columns=['road_length_category'], prefix='road', drop_first=True)
X_test_eng = pd.get_dummies(X_test_eng, columns=['road_length_category'], prefix='road', drop_first=True)

for col in X_train_eng.columns:
    if col not in X_test_eng.columns:
        X_test_eng[col] = 0

X_train_eng = X_train_eng.drop(columns=['local_authority_id'])
X_test_eng = X_test_eng.drop(columns=['local_authority_id'])
print(f"Final feature count: {X_train_eng.shape[1]}")

## 4. Feature Scaling (RobustScaler)
print("\n## 4. Feature Scaling (RobustScaler)")
scaler = RobustScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_eng), columns=X_train_eng.columns, index=X_train_eng.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_eng), columns=X_test_eng.columns, index=X_test_eng.index)
print("Features scaled using RobustScaler.")

## 5. Optimized Model Training and Evaluation
print("\n## 5. Optimized Model Training and Evaluation")

# Model 1: Gradient Boosting
print("\n1. Training Gradient Boosting (Optimized)...")
gb_model = GradientBoostingRegressor(n_estimators=150, learning_rate=0.005, max_depth=2, min_samples_split=300, min_samples_leaf=150, subsample=0.6, max_features='sqrt', validation_fraction=0.1, n_iter_no_change=20, random_state=42)
gb_model.fit(X_train_scaled, y_train)
gb_train_pred = gb_model.predict(X_train_scaled)
gb_test_pred = gb_model.predict(X_test_scaled)
gb_train_r2 = r2_score(y_train, gb_train_pred)
gb_test_r2 = r2_score(y_test, gb_test_pred)
gb_test_rmse = np.sqrt(mean_squared_error(y_test, gb_test_pred))
gb_test_mae = mean_absolute_error(y_test, gb_test_pred)
gb_cv_scores = cross_val_score(gb_model, X_train_scaled, y_train, cv=5, scoring='r2', n_jobs=-1)
print(f"  Train R²: {gb_train_r2:.4f}, Test R²: {gb_test_r2:.4f}, Test RMSE: {gb_test_rmse:.2f}, Overfitting gap: {gb_train_r2 - gb_test_r2:.4f}")

# Model 2: Ridge Regression
print("\n2. Training Ridge Regression...")
ridge_model = Ridge(alpha=1000.0, random_state=42)
ridge_model.fit(X_train_scaled, y_train)
ridge_train_r2 = r2_score(y_train, ridge_model.predict(X_train_scaled))
ridge_test_r2 = r2_score(y_test, ridge_model.predict(X_test_scaled))
print(f"  Train R²: {ridge_train_r2:.4f}, Test R²: {ridge_test_r2:.4f}, Overfitting gap: {ridge_train_r2 - ridge_test_r2:.4f}")

# Model 3: Lasso Regression
print("\n3. Training Lasso Regression...")
lasso_model = Lasso(alpha=1000.0, random_state=42, max_iter=5000)
lasso_model.fit(X_train_scaled, y_train)
lasso_train_r2 = r2_score(y_train, lasso_model.predict(X_train_scaled))
lasso_test_r2 = r2_score(y_test, lasso_model.predict(X_test_scaled))
print(f"  Train R²: {lasso_train_r2:.4f}, Test R²: {lasso_test_r2:.4f}, Overfitting gap: {lasso_train_r2 - lasso_test_r2:.4f}")

# Model 4: Random Forest
print("\n4. Training Random Forest (Constrained)...")
rf_model = RandomForestRegressor(n_estimators=100, max_depth=8, min_samples_split=150, min_samples_leaf=75, max_features='sqrt', max_samples=0.6, random_state=42, n_jobs=-1)
rf_model.fit(X_train_scaled, y_train)
rf_train_r2 = r2_score(y_train, rf_model.predict(X_train_scaled))
rf_test_r2 = r2_score(y_test, rf_model.predict(X_test_scaled))
print(f"  Train R²: {rf_train_r2:.4f}, Test R²: {rf_test_r2:.4f}, Overfitting gap: {rf_train_r2 - rf_test_r2:.4f}")

Traffic data loaded: (5529, 7)

Initial Data Overview

## 2. Train/Test Split (Preventing Leakage)
Train set: 4423 samples, Test set: 1106 samples

## 3. Feature Engineering and Encoding
1. Creating temporal features...
2. Creating road length features and bins...
3. Creating interaction features...
4. Creating authority frequency features...
5. Applying target encoding (CV + Smoothing)...
6. Final encoding and cleanup...
Final feature count: 16

## 4. Feature Scaling (RobustScaler)
Features scaled using RobustScaler.

## 5. Optimized Model Training and Evaluation

1. Training Gradient Boosting (Optimized)...
  Train R²: 0.5978, Test R²: 0.6400, Test RMSE: 950099258.48, Overfitting gap: -0.0422

2. Training Ridge Regression...
  Train R²: 0.9685, Test R²: 0.9638, Overfitting gap: 0.0046

3. Training Lasso Regression...
  Train R²: 0.9780, Test R²: 0.9455, Overfitting gap: 0.0325

4. Training Random Forest (Constrained)...
  Train R²: 0.8807, Test R²: 0.8844, Overfitting gap: -0.0036


In [7]:
import plotly.express as px
import pandas as pd

# Calculate Total Traffic per Year
df_time_series = df.groupby('year')['all_motor_vehicles'].sum().reset_index()
df_time_series.columns = ['Year', 'Total Traffic Volume']

fig_trend = px.line(
    df_time_series,
    x='Year',
    y='Total Traffic Volume',
    title='Total Annual Motor Vehicle Volume (1993-2019)',
    markers=True,
    labels={'Total Traffic Volume': 'Total Annual Motor Vehicles'}
)

fig_trend.update_layout(
    xaxis_tickangle=-45,
    hovermode='x unified'
)

fig_trend.show()

In [8]:
# Calculate Total Traffic per Local Authority
df_authority_rank = df.groupby('local_authority_name')['all_motor_vehicles'].sum().nlargest(15).reset_index()
df_authority_rank.columns = ['Local Authority', 'Total Traffic']

fig_bar = px.bar(
    df_authority_rank,
    x='Total Traffic',
    y='Local Authority',
    orientation='h',
    title='Top 15 Local Authorities by Total Traffic Volume',
    text_auto='.2s',
    color='Total Traffic',
    color_continuous_scale=px.colors.sequential.Teal
)

fig_bar.update_layout(yaxis={'categoryorder':'total ascending'})
fig_bar.update_traces(textposition='outside')
fig_bar.show()

In [10]:
df_sample = df.sample(n=10000, random_state=42) if len(df) > 10000 else df.copy()

fig_scatter = px.scatter(
    df_sample,
    x='link_length_km',
    y='all_motor_vehicles',
    title='Traffic Volume vs. Road Link Length (Sampled Data)',
    log_y=True, # Critical for handling the skewed target variable
    labels={
        'link_length_km': 'Road Link Length (km)',
        'all_motor_vehicles': 'Annual Motor Vehicle Count (Log Scale)'
    },
    # Use color to potentially show another feature, like year, for context
    color='year',
    opacity=0.5
)

fig_scatter.update_layout(
    xaxis_title='Road Link Length (km)',
    yaxis_title='Traffic Volume (Log Scale)',
    hovermode='closest'
)

fig_scatter.show()